In [1]:
import pandas as pd
import numpy as np
import pickle

In [82]:
raw_data = pd.read_csv('C:\\Users\\Russ\\Desktop\\Final_Project_Fall_2021\\accident_data.csv')

In [83]:
# The first column Un-named is useless so I am eliminating it here:
raw_data = raw_data.iloc[:,1:]

In [69]:
# The size of my entire dataset is:
print(raw_data.shape)

(61155, 47)


In [84]:
# I was assigned the environmental data for my group project so specifically these variables:
env_vars = ['Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 
                    'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 
                    'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']

In [85]:
# Keeps only the environmental variables in my dataframe:
for i in raw_data:
    if i in env_vars:
        continue
    else: 
        raw_data = raw_data.drop(i, axis = 1)
        
print(raw_data)

      Weather_Timestamp  Temperature(F)  Wind_Chill(F)  Humidity(%)  \
0       3/22/2016 19:47            64.0            NaN         24.0   
1       3/22/2016 20:53            64.0            NaN         23.0   
2        3/23/2016 7:47            55.9            NaN         53.0   
3       3/23/2016 11:47            73.9            NaN         14.0   
4       3/23/2016 11:47            73.9            NaN         14.0   
...                 ...             ...            ...          ...   
61150  12/10/2019 15:52            65.0           65.0         63.0   
61151  12/10/2019 15:52            65.0           65.0         63.0   
61152  12/10/2019 16:53            62.0           62.0         53.0   
61153  12/10/2019 16:52            64.0           64.0         70.0   
61154  12/10/2019 16:52            64.0           64.0         70.0   

       Pressure(in)  Visibility(mi) Wind_Direction  Wind_Speed(mph)  \
0             30.00            10.0            NNW              9.2   
1    

In [72]:
print('Percent of missing rows by column', '\n\n', raw_data.isnull().sum()/len(raw_data))

Percent of missing rows by column 

 Weather_Timestamp        0.006704
Temperature(F)           0.008863
Wind_Chill(F)            0.295364
Humidity(%)              0.009370
Pressure(in)             0.008617
Visibility(mi)           0.008846
Wind_Direction           0.017529
Wind_Speed(mph)          0.075497
Precipitation(in)        0.287466
Weather_Condition        0.008356
Sunrise_Sunset           0.000000
Civil_Twilight           0.000000
Nautical_Twilight        0.000000
Astronomical_Twilight    0.000000
dtype: float64


In [86]:
# After some trial and error I found that I can afford to remove any row 
# that has more than 2 missing values without losing very much data:
raw_data['number_missing'] = raw_data.isnull().sum(axis=1)
df_filtered = raw_data[raw_data['number_missing'] < 3]

In [74]:
df_filtered.shape

(56668, 15)

In [76]:
# Wind Chill and Precipitation still have a lot of missing data (ie about 24% for each):
# Because there are values that equal 0 in precipitation I am unsure whether NaN means
# precipitation = 0. I am also unclear with with Wind Chill 
print('Percent of missing rows by column', '\n\n', df_filtered.isnull().sum()/len(df_filtered))

Percent of missing rows by column 

 Weather_Timestamp        0.000000
Temperature(F)           0.000000
Wind_Chill(F)            0.239571
Humidity(%)              0.000547
Pressure(in)             0.000071
Visibility(mi)           0.001129
Wind_Direction           0.000018
Wind_Speed(mph)          0.003265
Precipitation(in)        0.243241
Weather_Condition        0.000565
Sunrise_Sunset           0.000000
Civil_Twilight           0.000000
Nautical_Twilight        0.000000
Astronomical_Twilight    0.000000
number_missing           0.000000
dtype: float64


In [87]:
# I will assume that if windchill is NaN then I will set it equal to the temperature: 
df_filtered['Wind_Chill(F)'].fillna(df_filtered['Temperature(F)'], inplace=True)

C:\Users\Russ\anaconda3\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [88]:
# All other missing numeric datatypes will be set to the median. 
# This works out so that days where precipitation was undefined are assumed 
# to be days with no rain. 
df_num = df_filtered.select_dtypes(include=np.number)

for i in df_filtered:
    if i in df_num:
        df_filtered.loc[df_filtered.loc[:,i].isnull(),i] = df_num.loc[:,i].median()
    else:
        continue
        
        


C:\Users\Russ\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [91]:
df_filtered.shape

(56668, 15)

In [92]:
df_filtered.head()

,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,number_missing
0,3/22/2016 19:47,64.0,64.0,24.0,30.00,10.0,NNW,9.2,0.0,Clear,Night,Night,Day,Day,2
1,3/22/2016 20:53,64.0,64.0,23.0,30.02,10.0,NNW,11.5,0.0,Clear,Night,Night,Night,Night,2
5,3/23/2016 12:53,75.0,75.0,13.0,30.16,10.0,West,10.4,0.0,Clear,Day,Day,Day,Day,2
6,3/23/2016 13:47,75.0,75.0,18.0,30.13,10.0,Variable,3.5,0.0,Clear,Day,Day,Day,Day,2
7,3/23/2016 13:47,75.0,75.0,18.0,30.13,10.0,Variable,3.5,0.0,Clear,Day,Day,Day,Day,2


In [100]:
print('Percent of missing rows by column', '\n\n', df_filtered.isnull().sum()/len(df_filtered))

Percent of missing rows by column 

 Weather_Timestamp        0.0
Temperature(F)           0.0
Wind_Chill(F)            0.0
Humidity(%)              0.0
Pressure(in)             0.0
Visibility(mi)           0.0
Wind_Direction           0.0
Wind_Speed(mph)          0.0
Precipitation(in)        0.0
Weather_Condition        0.0
Sunrise_Sunset           0.0
Civil_Twilight           0.0
Nautical_Twilight        0.0
Astronomical_Twilight    0.0
dtype: float64


In [99]:
# We can simply eliminate the very few rows that still have missing data
# Also we no longer need 'number_missing':
df_filtered = df_filtered.iloc[:,0:-1].dropna()

In [101]:
df_filtered.shape

(56635, 14)